In [ ]:
import time
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

# Initialize SageMaker session
sagemaker_session = sagemaker.Session()

# Define the IAM role with SageMaker permissions
role = 'ENTER ROLE HERE'

# Specify model data location (path to the .tar.gz file)
model_data = 'ENTER S3 BUCKET MODEL TAR GZ LOCATION HERE'

# Create the HuggingFace model object
huggingface_model = HuggingFaceModel(
    model_data=model_data,
    role=role,
    entry_point='inference.py',
    source_dir='.',  # Directory containing the inference script
    transformers_version='4.37.0',  
    pytorch_version='2.1.0',       
    py_version='py310'
)

try:
    # Deploy the model to create the endpoint
    print("Starting model deployment...")
    predictor = huggingface_model.deploy(
        initial_instance_count=1,
        instance_type='ml.m5.large',  # Use a valid CPU instance type
        endpoint_name='marcMLT5'
    )
    print("Model deployed successfully.")

    # Wait for the endpoint to be in service
    endpoint_name = predictor.endpoint_name
    client = sagemaker_session.sagemaker_client

    while True:
        response = client.describe_endpoint(EndpointName=endpoint_name)
        status = response['EndpointStatus']
        if status == 'InService':
            break
        elif status == 'Failed':
            raise Exception(f"Endpoint creation failed with status: {status}")
        print("Waiting for endpoint to be in service...")
        time.sleep(30)
    
    print(f"Endpoint {endpoint_name} is in service.")
    print(f"Endpoint name: {endpoint_name}")

except Exception as e:
    print(f"Failed to deploy model: {str(e)}")

# Check endpoint status directly after attempting to deploy
try:
    client = sagemaker_session.sagemaker_client
    response = client.describe_endpoint(EndpointName='marcMLT5')
    status = response['EndpointStatus']
    print(f"Direct endpoint status check: {status}")
    if status == 'InService':
        print(f"Endpoint marcMLT5 is in service.")
        print(f"Endpoint name: marcMLT5")
    elif status == 'Failed':
        raise Exception(f"Endpoint creation failed with status: {status}")
    else:
        print("Endpoint creation is still in progress.")
except Exception as e:
    print(f"Failed to check endpoint status: {str(e)}")

In [ ]:
import boto3
import json

region_name = 'us-east-1'
endpoint_name = 'marcMLT5'

sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=region_name)

def invoke_model(input_text):
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=json.dumps({'inputs': input_text})
    )
    result = json.loads(response['Body'].read().decode())
    return result

input_text = 955201
output = invoke_model(input_text)
print(output)


In [ ]:
#CLEANUP

import boto3

# Initialize SageMaker client
sagemaker_client = boto3.client('sagemaker')

def delete_endpoints():
    # List all endpoints
    endpoints = sagemaker_client.list_endpoints()['Endpoints']
    for endpoint in endpoints:
        endpoint_name = endpoint['EndpointName']
        try:
            sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
            print(f"Deleted endpoint: {endpoint_name}")
        except Exception as e:
            print(f"Error deleting endpoint {endpoint_name}: {e}")

def delete_endpoint_configs():
    # List all endpoint configurations
    endpoint_configs = sagemaker_client.list_endpoint_configs()['EndpointConfigs']
    for config in endpoint_configs:
        config_name = config['EndpointConfigName']
        try:
            sagemaker_client.delete_endpoint_config(EndpointConfigName=config_name)
            print(f"Deleted endpoint config: {config_name}")
        except Exception as e:
            print(f"Error deleting endpoint config {config_name}: {e}")

def delete_models():
    # List all models
    models = sagemaker_client.list_models()['Models']
    for model in models:
        model_name = model['ModelName']
        try:
            sagemaker_client.delete_model(ModelName=model_name)
            print(f"Deleted model: {model_name}")
        except Exception as e:
            print(f"Error deleting model {model_name}: {e}")

# Delete all endpoints
delete_endpoints()

# Delete all endpoint configurations
delete_endpoint_configs()

# Delete all models
delete_models()
